In [3]:
import requests
import matplotlib as mpl
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import geopandas as gpd
# from shapely.geometry import Polygon
from matplotlib import cm
import numpy as np
pd.set_option('future.no_silent_downcasting', True)
import os
import gzip
import shutil

In [4]:

def find_repo_root(start_path):
    current_path = os.path.abspath(start_path)
    
    while True:
        # Check for the existence of the .git directory or other indicators
        if os.path.isdir(os.path.join(current_path, '.git')) or \
           os.path.isfile(os.path.join(current_path, 'README.md')):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        
        # Stop if we reach the root directory
        if parent_path == current_path:
            break
        
        current_path = parent_path

    return None  # Return None if not found

root = find_repo_root(os.getcwd())
root

'c:\\Users\\fitsl\\Documents\\Programming\\UVM Programming Classes\\Data Science I\\Project'

In [5]:
url = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/" 
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">

<html>
<head>
<title>Index of /pub/data/swdi/stormevents/csvfiles</title>
</head>
<body>
<h1>Index of /pub/data/swdi/stormevents/csvfiles</h1>
<table>
<tr><th><a href="?C=N;O=D">Name</a></th><th><a href="?C=M;O=A">Last modified</a></th><th><a href="?C=S;O=A">Size</a></th><th><a href="?C=D;O=A">Description</a></th></tr>
<tr><th colspan="4"><hr/></th></tr>
<tr><td><a href="/pub/data/swdi/stormevents/">Parent Directory</a></td><td> </td><td align="right">  - </td><td> </td></tr>
<tr><td><a href="Storm-Data-Bulk-csv-Format.pdf">Storm-Data-Bulk-csv-Format.pdf</a></td><td align="right">2024-07-30 10:02  </td><td align="right">144K</td><td> </td></tr>
<tr><td><a href="Storm-Data-Export-Format.pdf">Storm-Data-Export-Format.pdf</a></td><td align="right">2024-07-30 10:02  </td><td align="right">147K</td><td> </td></tr>
<tr><td><a href="StormEvents_details-ftp_v1.0_d1950_c20210803.csv.gz">StormEvents_details-ftp_v1.0_d1950_c20210803.csv.gz<

In [6]:
# # get an object to represent the website contents
# url = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/" 
# r = requests.get(url)
# soup = BeautifulSoup(r.text, 'html.parser')


# for link in soup.find_all('a'):
#     href = link.get('href')
#     if href and href.endswith('.csv.gz'):
#         xls_url = urljoin(url, href)
#         print(f"Downloading: {xls_url}")  # Debug output
#         xls_response = requests.get(xls_url)
#         filename = os.path.join(root, "Data/StormEvents/Raw", os.path.basename(xls_url))
#         with open(filename, 'wb') as f:
#             f.write(xls_response.content)
# print("Downloaded all csv.gz files.")

In [7]:
# for filename in os.listdir(f"{root}/Data/StormEvents/Raw"):
#     print(filename)
#     new_filename = os.path.splitext(filename)[0]
#     with gzip.open(f"{root}/Data/StormEvents/Raw/{filename}", 'rb') as f_in:
#         with open(f"{root}/Data/StormEvents/Processed/{new_filename}", 'wb') as f_out:
#             shutil.copyfileobj(f_in, f_out)

In [8]:
df = pd.read_csv(f"{root}/Data/StormEvents/Processed/StormEvents_details-ftp_v1.0_d1975_c20220425.csv")
df.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [9]:
df[['CZ_FIPS', 'STATE_FIPS']]


,CZ_FIPS,STATE_FIPS
0,13,8
1,17,24
2,21,29
3,77,29
4,81,26
...,...,...
4970,485,48
4971,485,48
4972,111,12
4973,11,12


In [10]:
df['FULL_FIPS'] = df['STATE_FIPS'].map(lambda x : str(x) +"000")
df['FULL_FIPS'] = df['FULL_FIPS'].map(lambda x: x if len(x)==5 else '0' + x)

In [11]:
df['FULL_FIPS']

0       08000
1       24000
2       29000
3       29000
4       26000
        ...  
4970    48000
4971    48000
4972    12000
4973    12000
4974    05000
Name: FULL_FIPS, Length: 4975, dtype: object

In [12]:
fips = pd.read_csv(f"{root}/Code/Cleaning/Storm Data Cleaning/fips.csv", dtype={'code' : str})
fips['code']

0       01000
1       01001
2       01003
3       01005
4       01007
        ...  
3190    56037
3191    56039
3192    56041
3193    56043
3194    56045
Name: code, Length: 3195, dtype: object

In [24]:
df1 = pd.read_parquet(f"{root}/Data/Storm events/events_db_chunk_1.parquet")
df2 = pd.read_parquet(f"{root}/Data/Storm events/events_db_chunk_2.parquet")
df3 = pd.read_parquet(f"{root}/Data/Storm events/events_db_chunk_3.parquet")
df4 = pd.read_parquet(f"{root}/Data/Storm events/events_db_chunk_4.parquet")

df = pd.concat([df1, df2, df3, df4])


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

Thinking about how best to communicate this. Seems pretty simple to merge onto the county-by-count stuff. We could do something like bin the storm events into certain levels of 'intensity' based on (property damage, crop damage, people injured, people killed), and then put size-scaled dots into the counties on the map. Maybe could also do colored dots for whether its a tornado, hurricane, etc. This might be easier to do in R. 